In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np

init_cash = 350
path = '../data/APEUSDT_5m.csv'
symbol = path.split('/')[-1].split('_')[0].replace('USDT', '')
df = pd.read_csv(path)
df.head()

,time,open,high,low,close,volume
0,2023-08-01 00:00:00,1.856,1.858,1.855,1.857,198274.0
1,2023-08-01 00:05:00,1.856,1.856,1.850,1.856,328561.0
2,2023-08-01 00:10:00,1.855,1.859,1.855,1.857,206188.0
3,2023-08-01 00:15:00,1.857,1.861,1.856,1.859,253023.0
4,2023-08-01 00:20:00,1.859,1.864,1.858,1.861,394428.0


In [2]:
#Enabler
df['BELTHOLD'] = df.ta.cdl_pattern(name="belthold")
#Support
df['ADX'] = df.ta.adx()['ADX_14']
#Signal buy: BELTHOLD == 100, sell: BELTHOLD == -100
df['Signal_bull'] = np.where(df['BELTHOLD'] == 100, 2, 0)
df['Signal_bear'] = np.where(df['BELTHOLD'] == -100, 1, 0)
df['Signal'] = df['Signal_bull'] + df['Signal_bear']

#TP and SL indicator
df['ATR'] = df.ta.atr()

In [3]:
df.ta.cmo().describe()

count    23486.000000
mean        -0.699488
std         20.925355
min        -86.958634
25%        -14.695755
50%         -0.958290
75%         13.028293
max         84.397963
Name: CMO_14, dtype: float64

In [4]:
help(ta.td_seq)

Help on function td_seq in module pandas_ta.momentum.td_seq:

td_seq(close, asint=None, offset=None, **kwargs)
    TD Sequential (TD_SEQ)
    
    Tom DeMark's Sequential indicator attempts to identify a price point where an
    uptrend or a downtrend exhausts itself and reverses.
    
    Sources:
        https://tradetrekker.wordpress.com/tdsequential/
    
    Calculation:
        Compare current close price with 4 days ago price, up to 13 days. For the
        consecutive ascending or descending price sequence, display 6th to 9th day
        value.
    
    Args:
        close (pd.Series): Series of 'close's
        asint (bool): If True, fillnas with 0 and change type to int. Default: False
        offset (int): How many periods to offset the result. Default: 0
    
    Kwargs:
        show_all (bool): Show 1 - 13. If set to False, show 6 - 9. Default: True
        fillna (value, optional): pd.DataFrame.fillna(value)
    
    Returns:
        pd.DataFrame: New feature generated.



In [5]:
df['Signal'].value_counts()

0    18911
1     2358
2     2231
Name: Signal, dtype: int64

In [6]:
#dropna
df = df.dropna()
df.head()

,time,open,high,low,close,volume,BELTHOLD,ADX,Signal_bull,Signal_bear,Signal,ATR
27,2023-08-01 02:15:00,1.786,1.796,1.783,1.794,1389156.0,0.0,63.107162,0,0,0,0.014230
28,2023-08-01 02:20:00,1.794,1.797,1.788,1.788,833270.0,0.0,66.252199,0,0,0,0.013856
29,2023-08-01 02:25:00,1.789,1.794,1.785,1.785,613007.0,0.0,68.985941,0,0,0,0.013509
30,2023-08-01 02:30:00,1.785,1.792,1.780,1.783,900909.0,0.0,71.394043,0,0,0,0.013401
31,2023-08-01 02:35:00,1.783,1.786,1.779,1.783,492355.0,0.0,73.508381,0,0,0,0.012944


In [7]:
#set time to index

df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head()

C:\Users\hiban\AppData\Local\Temp\ipykernel_7548\1852176384.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])


,open,high,low,close,volume,BELTHOLD,ADX,Signal_bull,Signal_bear,Signal,ATR
time,,,,,,,,,,,
2023-08-01 02:15:00,1.786,1.796,1.783,1.794,1389156.0,0.0,63.107162,0,0,0,0.014230
2023-08-01 02:20:00,1.794,1.797,1.788,1.788,833270.0,0.0,66.252199,0,0,0,0.013856
2023-08-01 02:25:00,1.789,1.794,1.785,1.785,613007.0,0.0,68.985941,0,0,0,0.013509
2023-08-01 02:30:00,1.785,1.792,1.780,1.783,900909.0,0.0,71.394043,0,0,0,0.013401
2023-08-01 02:35:00,1.783,1.786,1.779,1.783,492355.0,0.0,73.508381,0,0,0,0.012944


In [8]:
#rename columns
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

In [9]:
df['Signal'].value_counts()

0    18886
1     2356
2     2231
Name: Signal, dtype: int64

In [10]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL(df,col_name):
    return df[col_name]

class Scalping_Strategy(Strategy):
    atr_sl_rate = 1.3
    TPSL_rate = 1.5
    adx_th = 25
    save_size_rate = 0.125
    init_cash = 350
    profit_target = 500
    size_multiplier_rate = 0.35

    def init(self):
        super().init()
        self.signal = self.I(SIGNAL, self.data, 'Signal')

    def next(self):
        super().next()

        slatr = self.atr_sl_rate * self.data.ATR[-1]
        size_multiplier = self.size_multiplier_rate if self.equity >= self.profit_target else 1.0

        if self.signal == 2 and self.data.ADX[-1] > self.adx_th:
            sl_price = self.data.Close[-1] - slatr
            tp_price = self.data.Close[-1] + self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity > self.init_cash * 1.5:
                self.buy(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.buy(sl=sl_price, tp=tp_price)

        if self.signal == 1 and self.data.ADX[-1] > self.adx_th:
            sl_price = self.data.Close[-1] + slatr
            tp_price = self.data.Close[-1] - self.TPSL_rate * slatr
            size = min(self.save_size_rate * self.equity / slatr, self.init_cash / slatr) * size_multiplier
            if self.equity >= self.init_cash * 1.5:
                self.sell(sl=sl_price, tp=tp_price, size=int(size))
            else:
                self.sell(sl=sl_price, tp=tp_price)
            

bt = Backtest(df, Scalping_Strategy, cash=init_cash, commission=.00002, margin=.05)
stat = bt.run()
stat

Start                     2023-08-01 02:15:00
End                       2023-10-21 14:15:00
Duration                     81 days 12:00:00
Exposure Time [%]                   37.281132
Equity Final [$]                    10.060511
Equity Peak [$]                   1577.241097
Return [%]                         -97.125568
Buy & Hold Return [%]              -37.513935
Return (Ann.) [%]                  -99.999999
Volatility (Ann.) [%]               46.313767
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.496305
Avg. Drawdown [%]                  -10.723189
Max. Drawdown Duration       59 days 14:30:00
Avg. Drawdown Duration        2 days 10:26:00
# Trades                                 1421
Win Rate [%]                        37.016186
Best Trade [%]                        1.90301
Worst Trade [%]                     -1.499835
Avg. Trade [%]                    

In [11]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stat['_equity_curve'].index, y=stat['_equity_curve']['Equity'])])
fig.show()

In [12]:
%%time

                    # size_multiplier_rate = np.arange(0.1, 0.5, 0.1).tolist(),
stats = bt.optimize(atr_sl_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    TPSL_rate = np.arange(0.3, 2.6, 0.2).tolist(),
                    adx_th = np.arange(10, 40, 5).tolist(),
                    maximize='Return [%]')
stats

C:\Users\hiban\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning:

Searching for best of 864 configurations.



  0%|          | 0/12 [00:00<?, ?it/s]

CPU times: total: 7min 48s
Wall time: 11min 13s


Start                     2023-08-01 02:15:00
End                       2023-10-21 14:15:00
Duration                     81 days 12:00:00
Exposure Time [%]                   65.428365
Equity Final [$]                  3132.814942
Equity Peak [$]                   5508.816845
Return [%]                         795.089984
Buy & Hold Return [%]              -37.513935
Return (Ann.) [%]               424564.770123
Volatility (Ann.) [%]     14363584118721.0...
Sharpe Ratio                              0.0
Sortino Ratio                     1569.082373
Calmar Ratio                      4873.729625
Max. Drawdown [%]                   -87.11291
Avg. Drawdown [%]                   -9.554963
Max. Drawdown Duration       38 days 06:40:00
Avg. Drawdown Duration        1 days 04:06:00
# Trades                                 2264
Win Rate [%]                        34.540636
Best Trade [%]                       3.328791
Worst Trade [%]                     -1.738493
Avg. Trade [%]                    

In [13]:
stats.keys()

Index(['Start', 'End', 'Duration', 'Exposure Time [%]', 'Equity Final [$]',
       'Equity Peak [$]', 'Return [%]', 'Buy & Hold Return [%]',
       'Return (Ann.) [%]', 'Volatility (Ann.) [%]', 'Sharpe Ratio',
       'Sortino Ratio', 'Calmar Ratio', 'Max. Drawdown [%]',
       'Avg. Drawdown [%]', 'Max. Drawdown Duration', 'Avg. Drawdown Duration',
       '# Trades', 'Win Rate [%]', 'Best Trade [%]', 'Worst Trade [%]',
       'Avg. Trade [%]', 'Max. Trade Duration', 'Avg. Trade Duration',
       'Profit Factor', 'Expectancy [%]', 'SQN', '_strategy', '_equity_curve',
       '_trades'],
      dtype='object')

In [14]:
stats._strategy

<Strategy Scalping_Strategy(atr_sl_rate=1.5000000000000002,TPSL_rate=2.3,adx_th=20)>

In [17]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Scatter(x=stats['_equity_curve'].index, y=stats['_equity_curve']['Equity'],name='Equity')])
#add a initial cash line
fig.add_trace(go.Scatter(x=stats['_equity_curve'].index, y=[init_cash]*len(stats['_equity_curve'].index), name='Initial Cash'))
fig.update_layout(title='Profit |'+symbol, yaxis_title='USDT', xaxis_title='Date')
fig.show()

In [16]:
stats['_trades'].tail()

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
2259,-14539,23411,23436,1.109978,1.105000,72.372234,0.004485,2023-10-21 09:10:00,2023-10-21 11:15:00,0 days 02:05:00
2260,-13668,23423,23436,1.107978,1.105000,40.701117,0.002688,2023-10-21 10:10:00,2023-10-21 11:15:00,0 days 01:05:00
2261,-13328,23423,23439,1.107978,1.104000,53.016652,0.003590,2023-10-21 10:10:00,2023-10-21 11:30:00,0 days 01:20:00
2262,24321,23442,23444,1.108022,1.102963,-123.034953,-0.004566,2023-10-21 11:45:00,2023-10-21 11:55:00,0 days 00:10:00
2263,13500,23439,23466,1.104022,1.113502,127.976565,0.008587,2023-10-21 11:30:00,2023-10-21 13:45:00,0 days 02:15:00
